In [2]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error, mean_squared_error, r2_score

Разделим данные на тренировочную и тестовую выборки, обучим простую модель k-ближайших соседей с 3мя соседями и оценим её качество на тестовых данных по нескольким метрикам: точность (accuracy), точность (precision), полнота (recall) и F1-мера (F1 Score).

In [7]:
# Загрузка данных для классификации
data = load_breast_cancer()
X, y = data.data, data.target

# Разделение данных на тренировочную и тестовую выборки для классификации
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Бейзлайн модель без оптимизации
knn_classifier_baseline = KNeighborsClassifier(n_neighbors=3)
knn_classifier_baseline.fit(X_train, y_train)

# Оценка бейзлайн модели
y_pred_baseline = knn_classifier_baseline.predict(X_test)
accuracy_baseline = accuracy_score(y_test, y_pred_baseline)
precision_baseline = precision_score(y_test, y_pred_baseline)
recall_baseline = recall_score(y_test, y_pred_baseline)
f1_baseline = f1_score(y_test, y_pred_baseline)

print(f"Baseline KNN Classification Metrics:\nAccuracy: {accuracy_baseline}\nPrecision: {precision_baseline}\nRecall: {recall_baseline}\nF1 Score: {f1_baseline}")

Baseline KNN Classification Metrics:
Accuracy: 0.9298245614035088
Precision: 0.9315068493150684
Recall: 0.9577464788732394
F1 Score: 0.9444444444444444


Точно также разделяем данные на тренировочную и тестовую выборки, выполняем стандартизацию признаков, обучаем модель регрессии k-ближайших соседей с 3мя соседями и оцениваем её производительность на тестовых данных с использованием метрик средней абсолютной ошибки (MAE), среднеквадратичной ошибки (MSE) и R^2 (коэффициент детерминации). 

In [8]:
# Загрузка данных для регрессии
boston = fetch_california_housing()
X_boston, y_boston = boston.data, boston.target

# Разделение данных на тренировочную и тестовую выборки для регрессии
X_train_boston, X_test_boston, y_train_boston, y_test_boston = train_test_split(X_boston, y_boston, test_size=0.2, random_state=42)

# Стандартизация данных для регрессии
scaler = StandardScaler()
X_train_boston = scaler.fit_transform(X_train_boston)
X_test_boston = scaler.transform(X_test_boston)

# Создание и обучение модели KNN для регрессии
knn_regressor = KNeighborsRegressor(n_neighbors=3)
knn_regressor.fit(X_train_boston, y_train_boston)

# Оценка модели для регрессии
y_pred_boston = knn_regressor.predict(X_test_boston)
mae = mean_absolute_error(y_test_boston, y_pred_boston)
mse = mean_squared_error(y_test_boston, y_pred_boston)
r2 = r2_score(y_test_boston, y_pred_boston)

print(f"\nBaseline KNN Regression Metrics:\nMAE: {mae}\nMSE: {mse}\nR-squared: {r2}")


Baseline KNN Regression Metrics:
MAE: 0.4599198611111111
MSE: 0.4666634350517549
R-squared: 0.6438795499720962


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures

Создадим пайплайн для стандартизации данных и примененим KNN-классификатор, а затем использует метод GridSearchCV для поиска наилучших гиперпараметров по сетке параметров с перекрестной проверкой.

In [9]:
## Улучшенная модель с оптимизацией
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

param_grid = {
    'knn__n_neighbors': [3, 5, 7, 9, 11],
    'knn__weights': ['uniform', 'distance'],
    'knn__p': [1, 2]  # Манхэттенское или Евклидово расстояние
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Обучаем улучшенную модель
best_classifier = grid_search.best_estimator_

# Оценка улучшенной модели
y_pred_improved = best_classifier.predict(X_test)
accuracy_improved = accuracy_score(y_test, y_pred_improved)
precision_improved = precision_score(y_test, y_pred_improved)
recall_improved = recall_score(y_test, y_pred_improved)
f1_improved = f1_score(y_test, y_pred_improved)

print(f"Improved KNN Classification Metrics:\nAccuracy: {accuracy_improved}\nPrecision: {precision_improved}\nRecall: {recall_improved}\nF1 Score: {f1_improved}")

Improved KNN Classification Metrics:
Accuracy: 0.9649122807017544
Precision: 0.971830985915493
Recall: 0.971830985915493
F1 Score: 0.971830985915493


Создадим пайплайн, который включает в себя:

- Заполнение пропущенных значений с помощью медианы.
- Стандартизацию данных.
- Создание полиномиальных признаков второй степени.
- Применение KNeighborsRegressor.

Далее оптимизируем гиперпараметры модели KNN с помощью GridSearchCV и перекрестной проверки. Обучаим модель с наилучшими параметрами, найденными в ходе оптимизации.


In [10]:
# Загрузка данных для регрессии
boston = fetch_california_housing()
X_boston, y_boston = boston.data, boston.target

# Разделение данных на тренировочную и тестовую выборки
X_train_boston, X_test_boston, y_train_boston, y_test_boston = train_test_split(X_boston, y_boston, test_size=0.2, random_state=42)

# Пайплайн для регрессии
pipeline_boston = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('polynomial', PolynomialFeatures(degree=2, include_bias=False)),
    ('knn', KNeighborsRegressor())
])

# Подбор гиперпараметров для регрессии
param_grid_reg = {
    'knn__n_neighbors': [3, 5, 7],
    'knn__weights': ['uniform', 'distance'],
    'knn__p': [1, 2]  # Манхэттенское или Евклидово расстояние
}

grid_search_boston = GridSearchCV(pipeline_boston, param_grid_reg, cv=5, scoring='neg_mean_squared_error')
grid_search_boston.fit(X_train_boston, y_train_boston)

# Обучаем модель, используя лучшие гиперпараметры
best_regressor = grid_search_boston.best_estimator_

# Оценка модели для регрессии
y_pred_boston = best_regressor.predict(X_test_boston)
mae = mean_absolute_error(y_test_boston, y_pred_boston)
mse = mean_squared_error(y_test_boston, y_pred_boston)
r2 = r2_score(y_test_boston, y_pred_boston)

print(f"\nImproved KNN Regression Metrics:\nMAE: {mae}\nMSE: {mse}\nR-squared: {r2}")


Improved KNN Regression Metrics:
MAE: 0.41595939485911765
MSE: 0.38251670809373284
R-squared: 0.7080936452318685


Реализуем кастомный KNN для решения задачи классификации.

In [11]:
from collections import Counter


class CustomKNNClassifier:
    def __init__(self, n_neighbors=3):
        self.n_neighbors = n_neighbors

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)

    def _predict(self, x):
        # Вычисление расстояний от x до всех точек в обучающем наборе
        distances = [np.linalg.norm(x - x_train) for x_train in self.X_train]
        # Получение k ближайших точек
        k_indices = np.argsort(distances)[:self.n_neighbors]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        # Определение наиболее встречающегося класса
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

In [12]:
# Загрузка данных и разделение на обучающий и тестовый наборы
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение пользовательского KNN-классификатора
knn = CustomKNNClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

# Оценка
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Custom KNN Classification Metrics:\nAccuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

Custom KNN Classification Metrics:
Accuracy: 0.9298245614035088
Precision: 0.9315068493150684
Recall: 0.9577464788732394
F1 Score: 0.9444444444444444


Также реализуем кастомный KNN для решения задачи регрессии.

In [15]:
class CustomKNNRegressor:
    def __init__(self, n_neighbors=3):
        self.n_neighbors = n_neighbors
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)
    
    def _predict(self, x):
        # Вычисление расстояний от x до всех точек в обучающем наборе
        distances = [np.linalg.norm(x - x_train) for x_train in self.X_train]
        # Получение k ближайших точек
        k_indices = np.argsort(distances)[:self.n_neighbors]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        # Возвращаем среднее значение соседей (для регрессии)
        return np.mean(k_nearest_labels)


In [16]:
# Загрузка данных
boston = fetch_california_housing()
X_boston, y_boston = boston.data, boston.target

# Разделение данных на тренировочную и тестовую выборки
X_train_boston, X_test_boston, y_train_boston, y_test_boston = train_test_split(X_boston, y_boston, test_size=0.2, random_state=42)

# Создание и обучение кастомного KNN-регрессора
custom_knn_regressor = CustomKNNRegressor(n_neighbors=3)
custom_knn_regressor.fit(X_train_boston, y_train_boston)

# Предсказания и оценка модели
y_pred_boston = custom_knn_regressor.predict(X_test_boston)
mae = mean_absolute_error(y_test_boston, y_pred_boston)
mse = mean_squared_error(y_test_boston, y_pred_boston)
r2 = r2_score(y_test_boston, y_pred_boston)

print(f"\nCustom KNN Regression Metrics:\nMAE: {mae}\nMSE: {mse}\nR-squared: {r2}")


Custom KNN Regression Metrics:
MAE: 0.8304867425710595
MSE: 1.1694144088518572
R-squared: 0.10759585116572867


Немного расширим ранее созданный класс `CustomKNNClassifier`. Дополним:

- fit: стандартизирует тренировочные данные перед обучением модели.
- predict: стандартизирует тестовые данные перед выполнением предсказаний.

In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

class ImprovedCustomKNNClassifier(CustomKNNClassifier):
    def fit(self, X, y):
        # Стандартизация данных
        self.scaler = StandardScaler().fit(X)
        X_scaled = self.scaler.transform(X)
        super().fit(X_scaled, y)

    def predict(self, X):
        X_scaled = self.scaler.transform(X)
        return super().predict(X_scaled)

# Используем улучшенную версию
improved_knn = ImprovedCustomKNNClassifier(n_neighbors=3)
improved_knn.fit(X_train, y_train)
y_pred_improved = improved_knn.predict(X_test)

# Оценка
accuracy_improved = accuracy_score(y_test, y_pred_improved)
precision_improved = precision_score(y_test, y_pred_improved)
recall_improved = recall_score(y_test, y_pred_improved)
f1_improved = f1_score(y_test, y_pred_improved)

print(f"Improved Custom KNN Classification Metrics:\nAccuracy: {accuracy_improved}\nPrecision: {precision_improved}\nRecall: {recall_improved}\nF1 Score: {f1_improved}")

Improved Custom KNN Classification Metrics:
Accuracy: 0.9473684210526315
Precision: 0.9577464788732394
Recall: 0.9577464788732394
F1 Score: 0.9577464788732394


Также улучшим раннее написанный регрессор. Дополним:

Будем использовать StandardScaler для стандартизации признаков (среднее 0, стандартное отклонение 1), что помогает алгоритму KNN работать более эффективно на данных с разными масштабами.

Добавим параметр p, который позволяет выбрать степень нормы, используемой для вычисления расстояний (по умолчанию Евклидово расстояние p=2. Можно выбрать Манхэттенское расстояние с p=1 и другие)

In [19]:
class ImprovedCustomKNNRegressor:
    def __init__(self, n_neighbors=3, p=2):
        self.n_neighbors = n_neighbors
        self.p = p
        
    def fit(self, X, y):
        # Стандартизация данных
        self.scaler = StandardScaler().fit(X)
        self.X_train = self.scaler.transform(X)
        self.y_train = y
        
    def predict(self, X):
        X_scaled = self.scaler.transform(X)
        predictions = [self._predict(x) for x in X_scaled]
        return np.array(predictions)
    
    def _predict(self, x):
        # Вычисление расстояний от x до всех точек в обучающем наборе
        distances = [np.linalg.norm(x - x_train, ord=self.p) for x_train in self.X_train]
        # Получение k ближайших точек
        k_indices = np.argsort(distances)[:self.n_neighbors]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        # Возвращаем среднее значение соседей (для регрессии)
        return np.mean(k_nearest_labels)

In [18]:
# Загрузка данных
boston = fetch_california_housing()
X_boston, y_boston = boston.data, boston.target

# Разделение данных на тренировочную и тестовую выборки
X_train_boston, X_test_boston, y_train_boston, y_test_boston = train_test_split(X_boston, y_boston, test_size=0.2, random_state=42)

# Создание и обучение улучшенного кастомного KNN-регрессора
improved_knn_regressor = ImprovedCustomKNNRegressor(n_neighbors=3, p=2)
improved_knn_regressor.fit(X_train_boston, y_train_boston)

# Предсказания и оценка модели
y_pred_boston = improved_knn_regressor.predict(X_test_boston)
mae = mean_absolute_error(y_test_boston, y_pred_boston)
mse = mean_squared_error(y_test_boston, y_pred_boston)
r2 = r2_score(y_test_boston, y_pred_boston)

print(f"\nImproved Custom KNN Regression Metrics:\nMAE: {mae}\nMSE: {mse}\nR-squared: {r2}")


Improved Custom KNN Regression Metrics:
MAE: 0.4599198611111111
MSE: 0.4666634350517549
R-squared: 0.6438795499720962


**Вывод по первой лабораторной работе:**

- лучшая модель для решения задачи классификации при помощи алгоритма KNN - улучшенный бейзлайн из библиотеки.
- лучшая модель для решения задачи регрессии при помощи алгоритма KNN - улучшенная бейзлайн из библиотеки.